# Import Libraries

In [1]:
from swarmintelligence import MemoryBasedGreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools
import math
import random as rd

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# # Load the image
# citra_input = cv2.imread('datasets/Testing/Mandrill.tiff')
# b,g,r = cv2.split(citra_input)
# citra_input = cv2.merge((r,g,b))

In [3]:
# gwo = GreyWolfOptimization(
#     k=10,
#     wolfSize=25,
#     maxIteration=100,
#     fitness_function='otsu',
#     obj='max'
# )
# # Evaluation Metrics
# st = time.time()
# greyWolfs_b, best_thresholds_b = gwo.fit_run(b)
# greyWolfs_g, best_thresholds_g = gwo.fit_run(g)
# greyWolfs_r, best_thresholds_r = gwo.fit_run(r)
# et = time.time()

# regions_b = Utilization().digitize(b, best_thresholds_b)
# regions_g = Utilization().digitize(g, best_thresholds_g)
# regions_r = Utilization().digitize(r, best_thresholds_r)
# merge_regions = cv2.merge((regions_r,regions_g,regions_b))

In [4]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 1))

In [5]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [6]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
optimal_params

{'wolfSize': 25, 'maxIteration': 100, 'crossover_rate': 1.0}

# Evaluation

In [7]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [8]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs, best_thresholds = gwo.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(gwo.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(gwo.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                gwo_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions'].append(mean_regions)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['Fitness'].append(fitness_values)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
Mean Fitness = 1948.7181309881873
	 AirplaneF16.tiff | Threshold =  3
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.805475833105
2024.805475833105
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.7838782335534
2024.7871768318307
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
202

3254.63925514276
3253.7244801842826
3254.63925514276
Mean Fitness = 3254.535929096227
	 Mandrill.tiff | Threshold =  2
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
Mean Fitness = 1549.02713277089
	 Mandrill.tiff | Threshold =  3
1639.466171099567
1639.466171099567
1639.466171099567
1639.413602033513
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.463360998692
1639.466171099567
1639.4633609

4217.995099043379
4218.012743389364
4218.034079186658
4218.034079186658
4217.9521147484475
4218.034079186658
4217.983460413122
4217.983460413122
Mean Fitness = 4217.992655241151


## Visualize Results using DataFrame

In [9]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[113, 173]",1.129657,1948.718131,2270.227447,47.646898,14.570112,0.794424,0.940226,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.218674659729004, 1.0475709438323975, 0.9943...","[1948.7183902321062, 1948.7183902321062, 1948....","[2270.6405181884766, 2270.6405181884766, 2270....","[47.651238369936166, 47.651238369936166, 47.65...","[14.569319775617668, 14.569319775617668, 14.56...","[0.7944112258870609, 0.7944112258870609, 0.794...","[0.9402201034707982, 0.9402201034707982, 0.940..."
1,AirplaneF16.tiff,3,otsu,max,"[93, 145, 191]",1.110021,2024.827766,1773.460517,42.112208,15.642698,0.779065,0.953245,"[[93, 255, 255, 191, 191, 191, 255, 191, 255, ...","[1.1750526428222656, 1.128763198852539, 0.9727...","[2024.8375098105803, 2024.8375098105803, 2024....","[1774.3395538330078, 1774.3395538330078, 1774....","[42.12291008267363, 42.12291008267363, 42.1229...","[15.640436268576428, 15.640436268576428, 15.64...","[0.77934507049225, 0.77934507049225, 0.7793450...","[0.9532349424759257, 0.9532349424759257, 0.953..."
2,AirplaneF16.tiff,4,otsu,max,"[84, 129, 172, 203]",0.999825,2069.976436,1100.239374,33.168823,17.716469,0.732559,0.969284,"[[84, 203, 203, 203, 203, 203, 203, 203, 203, ...","[1.2383270263671875, 1.0900850296020508, 0.883...","[2069.932857317965, 2070.0794736113085, 2069.8...","[1091.6536865234375, 1094.045597076416, 1093.1...","[33.04018290693073, 33.07636009412789, 33.0624...","[17.74995475141993, 17.74044938188586, 17.7441...","[0.7321918575892461, 0.7326687134455045, 0.731...","[0.9696552295343954, 0.9693723689797689, 0.969..."
3,AirplaneF16.tiff,5,otsu,max,"[71, 108, 144, 180, 205]",0.838734,2095.894101,904.497442,30.074350,18.567030,0.745396,0.975408,"[[71, 205, 205, 205, 180, 205, 205, 205, 205, ...","[0.9278750419616699, 0.9405975341796875, 1.074...","[2096.0857299534687, 2096.1432770275533, 2096....","[906.4246559143066, 907.2876815795898, 905.764...","[30.106887184069805, 30.121216469120064, 30.09...","[18.55748650484443, 18.55335346450818, 18.5606...","[0.7450408811064716, 0.7452811482856196, 0.745...","[0.9753449854423007, 0.9753724479476479, 0.975..."
4,Lena.png,2,otsu,max,"[93, 151]",0.763458,1961.822013,2813.567070,53.043068,13.638231,0.612182,0.883309,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.6224963665008545, 0.9671077728271484, 0.726...","[1961.8220125153634, 1961.8220125153634, 1961....","[2813.567070007324, 2813.567070007324, 2813.56...","[53.04306806744237, 53.04306806744237, 53.0430...","[13.638230885124642, 13.638230885124642, 13.63...","[0.6121817460412936, 0.6121817460412936, 0.612...","[0.8833090574580458, 0.8833090574580458, 0.883..."
5,Lena.png,3,otsu,max,"[81, 127, 171]",0.770437,2128.299351,1250.170480,35.357745,17.161114,0.706230,0.933618,"[[171, 171, 171, 171, 171, 171, 171, 171, 171,...","[0.9127600193023682, 0.7186353206634521, 0.790...","[2128.284181016679, 2128.3048672466566, 2128.3...","[1252.4491691589355, 1249.3418655395508, 1249....","[35.389958592218434, 35.346030407098766, 35.34...","[17.153202518789254, 17.16399067408044, 17.163...","[0.7072313681226016, 0.7058656377663244, 0.705...","[0.9340987864621145, 0.9334425202424386, 0.933..."
6,Lena.png,4,otsu,max,"[75, 114, 145, 180]",0.767151,2191.869807,794.701711,28.190412,19.128788,0.753519,0.954114,"[[180, 180, 180, 180, 180, 180, 180, 180, 180,...","[0.7473845481872559, 0.7084462642669678, 0.719...","[2191.8485530059365, 2191.874720070565, 2191.8...","[791.031623840332, 795.6595458984375, 795.6595...","[28.12528442238997, 28.2074377762043, 28.20743...","[19.148865148104118, 19.12353083289055, 19.123...","[0.7540597418757801, 0.7534048142269026, 0.753...","[0.9549007300695751, 0.9539824804728553, 0.953..."
7,Lena.png,5,otsu,max,"[72, 107, 134, 158, 187]",1.032054,2217.500898

# Save results

In [10]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_otsu_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_otsu_dict.pkl', 'wb'))